# Шустров Максим Валерьевич

# Улицы. Объекты культурного наследия. Невского района

## PIP, Import, Константы, Функции

### PIP модулей

In [1]:
# todo собрать установку всех необходимых модулей в одном месте
!pip install folium -U
!pip install geopandas mapclassify osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import модулей

In [2]:
# todo собрать импорты всех модулей в одном месте
import pandas as pd
import geopandas as gpd
import osmnx as ox

### КОНСТАНТЫ

In [23]:
# Подложка карт. Выбрана "CartoDB positron", по умолчанию “OpenStreetMap” 
TILES = "CartoDB positron"

# Данные, с которыми будет работа. Район, улицы, памятники истоии и культуры
TERRITORY_NAME = 'Невский район, Санкт-Петербург'
STREETS_FILE_URL = "https://drive.google.com/file/d/1bUT1E-QSbG1vpSNM2dOG2-LEVXSrPdo3/view?usp=sharing"
KGIOP_FILE_URL = "https://raw.githubusercontent.com/m-shustrov/ITMO_LESSON/main/kgiop_objects.geojson"

STREET_BUFFER = 100 # Применяется для буфера улиц

### Функции

In [4]:
def get_google_drive_download_url(url: str) -> str:
    """
    Функция возвращает ссылку для скачивания с google drive.
     Например: "https://drive.google.com/file/d/1MWGOuqV76e0ubQOg8Ke0KTU3yGpkprHp/view?usp=sharing" ->
     "https://drive.google.com/uc?export=download&id=1MWGOuqV76e0ubQOg8Ke0KTU3yGpkprHp"
    """
    drive_id = url.split("/")[5]
    return f"https://drive.google.com/uc?export=download&id={drive_id}"

## Территория

### Загрузка территории из OSM (Extract), Невский район СПб

In [5]:
territory = ox.geocode_to_gdf(TERRITORY_NAME)  # подгружаем с OSM и получаем GeoDataFrame
territory.explore(tiles = TILES)

### Обработка данных с территорией (Transform)

In [6]:
# Удаляем все столбцы (кроме столбца geometry). Добавляем столбец "Название территории" 
territory.drop(territory.columns.difference(['geometry']), 1, inplace=True) 
territory["Название территории"] = "Невский район"
territory

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,geometry,Название территории
0,"POLYGON ((30.36331 59.91387, 30.36350 59.91337...",Невский район


## Улицы

### Загрузка файла с улицами из google disk (Extract)

In [7]:
url = get_google_drive_download_url(STREETS_FILE_URL)
gdf_streets = gpd.read_file(url, mask=territory)  # Отфильтровываем улицы по маске геометрии территории полученной ранее
gdf_streets

,type,id,tags,geometry
0,way,4454554,"{'foot': 'no', 'hgv': 'no', 'highway': 'tertia...","LINESTRING (3391641.514 8369024.600, 3391686.0..."
1,way,4455612,"{'addr:postcode': '193318', 'highway': 'tertia...","LINESTRING (3388065.876 8383198.675, 3388058.2..."
2,way,4456145,"{'foot': 'no', 'highway': 'primary', 'lanes': ...","LINESTRING (3391979.892 8382039.064, 3391986.1..."
3,way,4456665,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3381745.367 8378798.756, 3381756.9..."
4,way,4456737,"{'foot': 'no', 'highway': 'primary', 'lanes': ...","LINESTRING (3391760.525 8382110.538, 3391738.0..."
...,...,...,...,...
8923,way,982174776,"{'highway': 'service', 'surface': 'unpaved'}","LINESTRING (3387171.491 8370639.425, 3387147.9..."
8924,way,983845877,"{'highway': 'service', 'service': 'parking_ais...","LINESTRING (3390005.551 8377847.117, 3389996.3..."
8925,way,983845878,"{'highway': 'service', 'service': 'parking_ais...","LINESTRING (3389872.625 8377858.460, 3389955.5..."
8926,way,983845879,"{'highway': 'service', 'service': 'parking_ais...","LINESTRING (3389876.755 8377827.961, 3389973.3..."


### Обработка данных с улицами (Transform)

In [8]:
#json_normalize позволяет преобразовать вложенные структуры данных в json файле с таблицу
tags = pd.json_normalize(gdf_streets["tags"])
tags

,foot,hgv,highway,lanes,lit,maxspeed,name,name:en,postal_code,surface,...,maxspeed:forward,construction,old_name:ru,parking:lane:right:condition,abandoned,turn:lanes:forward,was:oneway:bicycle,was:oneway:psv,motorcar,turn:lanes:backward
0,no,no,tertiary,2,yes,RU:urban,улица Чернова,Chernova Street,192012,asphalt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,tertiary,2,yes,RU:urban,Зольная улица,NaN,NaN,asphalt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,no,NaN,primary,5,yes,RU:urban,Российский проспект,NaN,NaN,asphalt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,no,NaN,secondary,1,yes,RU:urban,Глухоозёрское шоссе,NaN,NaN,asphalt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,no,NaN,primary,3,yes,RU:urban,NaN,NaN,NaN,asphalt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,NaN,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,unpaved,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8924,NaN,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8925,NaN,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8926,NaN,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8928 entries, 0 to 8927
Data columns (total 80 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   foot                          959 non-null    object
 1   hgv                           30 non-null     object
 2   highway                       8928 non-null   object
 3   lanes                         1386 non-null   object
 4   lit                           1462 non-null   object
 5   maxspeed                      1448 non-null   object
 6   name                          1288 non-null   object
 7   name:en                       539 non-null    object
 8   postal_code                   152 non-null    object
 9   surface                       3772 non-null   object
 10  addr:postcode                 61 non-null     object
 11  wikidata                      322 non-null    object
 12  wikipedia                     325 non-null    object
 13  oneway            

In [10]:
#Из всех тегов оставим только тег "name".
extract_tags = [
    "name",
]
# И прицепим их к улицам с помощью метода join. По умолчанию сопоставление происходит по столбцу с индексами.
gdf_streets = gdf_streets.join(tags[extract_tags])
gdf_streets

,type,id,tags,geometry,name
0,way,4454554,"{'foot': 'no', 'hgv': 'no', 'highway': 'tertia...","LINESTRING (3391641.514 8369024.600, 3391686.0...",улица Чернова
1,way,4455612,"{'addr:postcode': '193318', 'highway': 'tertia...","LINESTRING (3388065.876 8383198.675, 3388058.2...",Зольная улица
2,way,4456145,"{'foot': 'no', 'highway': 'primary', 'lanes': ...","LINESTRING (3391979.892 8382039.064, 3391986.1...",Российский проспект
3,way,4456665,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3381745.367 8378798.756, 3381756.9...",Глухоозёрское шоссе
4,way,4456737,"{'foot': 'no', 'highway': 'primary', 'lanes': ...","LINESTRING (3391760.525 8382110.538, 3391738.0...",NaN
...,...,...,...,...,...
8923,way,982174776,"{'highway': 'service', 'surface': 'unpaved'}","LINESTRING (3387171.491 8370639.425, 3387147.9...",NaN
8924,way,983845877,"{'highway': 'service', 'service': 'parking_ais...","LINESTRING (3390005.551 8377847.117, 3389996.3...",NaN
8925,way,983845878,"{'highway': 'service', 'service': 'parking_ais...","LINESTRING (3389872.625 8377858.460, 3389955.5...",NaN
8926,way,983845879,"{'highway': 'service', 'service': 'parking_ais...","LINESTRING (3389876.755 8377827.961, 3389973.3...",NaN


In [11]:
gdf_streets.dropna(
    how="all",  # удалить если отсутствуют значения во всех указанных столбцах
    subset=["name"],  # столбцы, в которых надо искать пропуски
    inplace=True
)
gdf_streets

,type,id,tags,geometry,name
0,way,4454554,"{'foot': 'no', 'hgv': 'no', 'highway': 'tertia...","LINESTRING (3391641.514 8369024.600, 3391686.0...",улица Чернова
1,way,4455612,"{'addr:postcode': '193318', 'highway': 'tertia...","LINESTRING (3388065.876 8383198.675, 3388058.2...",Зольная улица
2,way,4456145,"{'foot': 'no', 'highway': 'primary', 'lanes': ...","LINESTRING (3391979.892 8382039.064, 3391986.1...",Российский проспект
3,way,4456665,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3381745.367 8378798.756, 3381756.9...",Глухоозёрское шоссе
5,way,4457777,"{'highway': 'secondary', 'lanes': '3', 'lit': ...","LINESTRING (3390430.324 8381152.697, 3390472.5...",улица Коллонтай
...,...,...,...,...,...
8870,way,977049605,"{'highway': 'residential', 'lanes': '2', 'lit'...","LINESTRING (3385580.101 8377041.069, 3385599.5...",улица Пинегина
8871,way,977049606,"{'foot': 'no', 'highway': 'tertiary', 'lanes':...","LINESTRING (3386196.365 8377309.305, 3386206.7...",улица Бабушкина
8872,way,977049607,"{'foot': 'no', 'highway': 'tertiary', 'lanes':...","LINESTRING (3386206.774 8377271.283, 3386231.7...",Большой Смоленский проспект
8873,way,977049608,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3394758.638 8381436.851, 3394767.8...",проспект Солидарности


In [12]:
# Проверяем наличие дубликатов по полям "id", "geometry", "name"
tag_duble = [
    "id", 
    "geometry", 
    "name"
]
gdf_streets.duplicated(subset=tag_duble).value_counts()

False    1288
dtype: int64

In [13]:
# Проверка, являются ли названия улиц уникальными.
gdf_streets["name"].is_unique

False

In [14]:
# Метод dissolve сгруппирует геометрии по столбцу "name" и объединит геометрии в одну
gdf_streets = gdf_streets.dissolve(by="name")
gdf_streets

,geometry,type,id,tags
name,,,,
1-й Рабфаковский переулок,"LINESTRING (3391743.672 8368417.242, 3391766.1...",way,132916332,"{'highway': 'residential', 'lanes': '1', 'lit'..."
1-я дорога,"LINESTRING (3393246.184 8372403.217, 3393268.7...",way,59291558,"{'highway': 'service', 'name': '1-я дорога'}"
11-я линия,"LINESTRING (3391094.356 8367013.072, 3391152.4...",way,774757869,"{'access': 'private', 'highway': 'service', 'n..."
13-я линия,"LINESTRING (3391016.210 8366855.634, 3391125.3...",way,774757865,"{'access': 'private', 'highway': 'service', 'n..."
2-й Рабфаковский переулок,"LINESTRING (3392568.994 8367091.715, 3392561.7...",way,173534236,"{'highway': 'residential', 'lanes': '1', 'lit'..."
...,...,...,...,...
улица Чернова,"MULTILINESTRING ((3391641.514 8369024.600, 339...",way,4454554,"{'foot': 'no', 'hgv': 'no', 'highway': 'tertia..."
улица Чудновского,"MULTILINESTRING ((3392437.637 8382877.735, 339...",way,8146822,"{'foot': 'no', 'highway': 'residential', 'lane..."
улица Шелгунова,"MULTILINESTRING ((3388527.362 8369046.328, 338...",way,31369845,"{'foot': 'no', 'highway': 'residential', 'lane..."


In [15]:
# Столбец "name" переиминуем в "Название улицы"
gdf_streets.index.rename("Название улицы", inplace=True)
gdf_streets.head()

,geometry,type,id,tags
Название улицы,,,,
1-й Рабфаковский переулок,"LINESTRING (3391743.672 8368417.242, 3391766.1...",way,132916332,"{'highway': 'residential', 'lanes': '1', 'lit'..."
1-я дорога,"LINESTRING (3393246.184 8372403.217, 3393268.7...",way,59291558,"{'highway': 'service', 'name': '1-я дорога'}"
11-я линия,"LINESTRING (3391094.356 8367013.072, 3391152.4...",way,774757869,"{'access': 'private', 'highway': 'service', 'n..."
13-я линия,"LINESTRING (3391016.210 8366855.634, 3391125.3...",way,774757865,"{'access': 'private', 'highway': 'service', 'n..."
2-й Рабфаковский переулок,"LINESTRING (3392568.994 8367091.715, 3392561.7...",way,173534236,"{'highway': 'residential', 'lanes': '1', 'lit'..."


In [16]:
# Удаляем столбцы "id", "type" и "tags"
drop_columns = [
    "id", 
    "type",
    "tags"
]
gdf_streets.drop(columns=drop_columns, inplace=True)
gdf_streets.head()

,geometry
Название улицы,
1-й Рабфаковский переулок,"LINESTRING (3391743.672 8368417.242, 3391766.1..."
1-я дорога,"LINESTRING (3393246.184 8372403.217, 3393268.7..."
11-я линия,"LINESTRING (3391094.356 8367013.072, 3391152.4..."
13-я линия,"LINESTRING (3391016.210 8366855.634, 3391125.3..."
2-й Рабфаковский переулок,"LINESTRING (3392568.994 8367091.715, 3392561.7..."


In [17]:
gdf_streets.explore(tiles=TILES)

## Объекты культурного наследия

### Загрузка объектов культурного наследия

In [18]:
#Загружаем объекты культупного наследия в соотвествии с нашей маской (Невский район)
gdf_kgiop_objects = gpd.read_file(KGIOP_FILE_URL, mask=territory)
gdf_kgiop_objects

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,4072,Палевский жилмассив (19 домов),Жилой дом,1925-1928,"Обуховской Обороны пр., 95, корп. 6",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3387453.441 8376910.050)
1,4051,—,Здание школы (с садом и оградой),1932-1933,"Большой Смоленский пр., 36, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Ансамбль,POINT (3385184.415 8376455.066)
2,3974,—,Октябрьская набережная,1932-1936,"На правом берегу р. Невы, от устья р. Утки до ...",объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Ансамбль,POINT (3388501.848 8377901.347)
3,3975,—,Терраса у завода им. Ломоносова,1926,На р. Неве по пр. Обуховской Обороны,объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Памятник,POINT (3389224.756 8373469.957)
4,4008,—,Шлиссельбургский мост,1928-1929,Через Обводный канал по Обуховской обороны пр.,объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Памятник,POINT (3383805.278 8381795.019)
...,...,...,...,...,...,...,...,...,...
244,9618,"Рабочий городок при ГРЭС ""Красный Октябрь"" (ТЭ...",Жилой дом,1926-1928,"Октябрьская наб., 94, корпус 3, лит. В",объект культурного наследия регионального знач...,Распоряжение КГИОП от 14.08.2019 № 491-р,Памятник,POINT (3392198.323 8371250.714)
245,9619,"Рабочий городок при ГРЭС ""Красный Октябрь"" (ТЭ...",Жилой дом,1926-1928,"Октябрьская наб., 96, корпус 1, лит. М",объект культурного наследия регионального знач...,Распоряжение КГИОП от 14.08.2019 № 491-р,Памятник,POINT (3392241.960 8371116.095)
246,9620,"Рабочий городок при ГРЭС ""Красный Октябрь"" (ТЭ...",Жилой дом,1926-1928,"Октябрьская наб., 96, корпус 2, лит. А",объект культурного наследия регионального знач...,Распоряжение КГИОП от 14.08.2019 № 491-р,Памятник,POINT (3392244.854 8371170.652)
247,9621,"Рабочий городок при ГРЭС ""Красный Октябрь"" (ТЭ...",Жилой дом,1926-1928,"Октябрьская наб., 96, корпус 3, лит. А",объект культурного наследия регионального знач...,Распоряжение КГИОП от 14.08.2019 № 491-р,Памятник,POINT (3392259.882 8371238.738)


In [19]:
# Оставляем только столбцы object_name, object_location, object_type
gdf_kgiop_objects.drop(gdf_kgiop_objects.columns.difference(['object_name','object_location','object_type', 'geometry']), 1, inplace=True)
# Проверяем
gdf_kgiop_objects

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,object_name,object_location,object_type,geometry
0,Жилой дом,"Обуховской Обороны пр., 95, корп. 6",Памятник,POINT (3387453.441 8376910.050)
1,Здание школы (с садом и оградой),"Большой Смоленский пр., 36, лит. А",Ансамбль,POINT (3385184.415 8376455.066)
2,Октябрьская набережная,"На правом берегу р. Невы, от устья р. Утки до ...",Ансамбль,POINT (3388501.848 8377901.347)
3,Терраса у завода им. Ломоносова,На р. Неве по пр. Обуховской Обороны,Памятник,POINT (3389224.756 8373469.957)
4,Шлиссельбургский мост,Через Обводный канал по Обуховской обороны пр.,Памятник,POINT (3383805.278 8381795.019)
...,...,...,...,...
244,Жилой дом,"Октябрьская наб., 94, корпус 3, лит. В",Памятник,POINT (3392198.323 8371250.714)
245,Жилой дом,"Октябрьская наб., 96, корпус 1, лит. М",Памятник,POINT (3392241.960 8371116.095)
246,Жилой дом,"Октябрьская наб., 96, корпус 2, лит. А",Памятник,POINT (3392244.854 8371170.652)
247,Жилой дом,"Октябрьская наб., 96, корпус 3, лит. А",Памятник,POINT (3392259.882 8371238.738)


In [20]:
# Переименовываем столбцы object_name, object_location, object_type в русские названия
tag_rename = {
    'object_name':'Наименование', 
    'object_location':'Локация', 
    'object_type':'Тип'
}
gdf_kgiop_objects.rename(columns=tag_rename, inplace=True)
# Проверяем
gdf_kgiop_objects

,Наименование,Локация,Тип,geometry
0,Жилой дом,"Обуховской Обороны пр., 95, корп. 6",Памятник,POINT (3387453.441 8376910.050)
1,Здание школы (с садом и оградой),"Большой Смоленский пр., 36, лит. А",Ансамбль,POINT (3385184.415 8376455.066)
2,Октябрьская набережная,"На правом берегу р. Невы, от устья р. Утки до ...",Ансамбль,POINT (3388501.848 8377901.347)
3,Терраса у завода им. Ломоносова,На р. Неве по пр. Обуховской Обороны,Памятник,POINT (3389224.756 8373469.957)
4,Шлиссельбургский мост,Через Обводный канал по Обуховской обороны пр.,Памятник,POINT (3383805.278 8381795.019)
...,...,...,...,...
244,Жилой дом,"Октябрьская наб., 94, корпус 3, лит. В",Памятник,POINT (3392198.323 8371250.714)
245,Жилой дом,"Октябрьская наб., 96, корпус 1, лит. М",Памятник,POINT (3392241.960 8371116.095)
246,Жилой дом,"Октябрьская наб., 96, корпус 2, лит. А",Памятник,POINT (3392244.854 8371170.652)
247,Жилой дом,"Октябрьская наб., 96, корпус 3, лит. А",Памятник,POINT (3392259.882 8371238.738)


In [21]:
gdf_kgiop_objects.explore(tiles=TILES)

In [24]:
def get_contains_kgiop_objects(street) -> int:
    """ Функция для подсчета количества объектов культурного наследия попадающих в буффер улицы"""
    return sum(gdf_kgiop_objects["geometry"].within(street.buffer(STREET_BUFFER)))


gdf_streets["contains_kgiop_objects"] = gdf_streets["geometry"].apply(get_contains_kgiop_objects)
gdf_streets.nlargest(5, "contains_kgiop_objects")

,geometry,contains_kgiop_objects
Название улицы,,
проспект Обуховской Обороны,"MULTILINESTRING ((3389659.225 8372593.936, 338...",30
Октябрьская набережная,"MULTILINESTRING ((3390229.059 8372810.613, 339...",19
Ивановская улица,"MULTILINESTRING ((3387949.614 8371426.765, 338...",10
улица Бабушкина,"MULTILINESTRING ((3392909.164 8368574.958, 339...",10
улица Ткачей,"MULTILINESTRING ((3388204.959 8376339.615, 338...",10


In [25]:
gdf_streets.explore("contains_kgiop_objects", tiles=TILES)

In [26]:
# Некая статистика
gdf_streets["density"] = gdf_streets["contains_kgiop_objects"] / gdf_streets.length
gdf_streets.nlargest(5, "density")

,geometry,contains_kgiop_objects,density
Название улицы,,,
площадь Культуры,"MULTILINESTRING ((3387118.792 8375902.398, 338...",5,0.011250
переулок Ногина,"LINESTRING (3386979.899 8377292.902, 3386990.9...",4,0.010771
Шлиссельбургский мост,"MULTILINESTRING ((3383839.965 8381783.026, 338...",1,0.007515
улица Ткачей,"MULTILINESTRING ((3388204.959 8376339.615, 338...",10,0.004358
Стеклянная улица,"LINESTRING (3384618.055 8380365.517, 3384640.6...",1,0.002366


In [28]:
# Переименовываем столбцы contains_kgiop_objects и density в русские названия
gdf_streets.rename(columns={'contains_kgiop_objects':'Кол-во КГИОП объектов', 'density':'плотность'}, inplace=True)
gdf_streets

,geometry,Кол-во КГИОП объектов,плотность
Название улицы,,,
1-й Рабфаковский переулок,"LINESTRING (3391743.672 8368417.242, 3391766.1...",0,0.0
1-я дорога,"LINESTRING (3393246.184 8372403.217, 3393268.7...",0,0.0
11-я линия,"LINESTRING (3391094.356 8367013.072, 3391152.4...",0,0.0
13-я линия,"LINESTRING (3391016.210 8366855.634, 3391125.3...",0,0.0
2-й Рабфаковский переулок,"LINESTRING (3392568.994 8367091.715, 3392561.7...",0,0.0
...,...,...,...
улица Чернова,"MULTILINESTRING ((3391641.514 8369024.600, 339...",0,0.0
улица Чудновского,"MULTILINESTRING ((3392437.637 8382877.735, 339...",0,0.0
улица Шелгунова,"MULTILINESTRING ((3388527.362 8369046.328, 338...",0,0.0


In [29]:
m = gdf_streets.explore("Кол-во КГИОП объектов", tiles=TILES)
m = territory.explore(m=m, style_kwds={"fill": False, "weight": 5})

m

# Сохранение датафрейма в формате geojson

In [33]:
# Cохраняем датафрейм gdf_streets в файл streets_with_contains_kgiop.geojson в формате geojson
gdf_streets.to_crs("epsg:4326").to_file('streets_with_contains_kgiop.geojson', driver='GeoJSON')
gdf_streets.to_crs("epsg:4326").to_file('4326_streets_with_contains_kgiop.geojson', driver='GeoJSON')